In [2]:
import pandas as pd
import os 
from dotenv import load_dotenv

In [3]:

# Load environment variables
load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [4]:
df = pd.read_csv("../data/game-dataset.csv",sep = ',')
df.columns

Index(['gameId', 'gameName', 'alternateNames', 'subcategory', 'level',
       'description', 'playersMax', 'ageRange', 'duration', 'equipmentNeeded',
       'objective', 'skillsDeveloped', 'setupTime', 'place',
       'physicalIntensityLevel', 'educationalBenefits', 'category'],
      dtype='object')

In [5]:
documents = df.to_dict(orient='records')
documents[0]

{'gameId': 1,
 'gameName': 'Soccer',
 'alternateNames': 'Football',
 'subcategory': 'Team Sport',
 'level': 'Beginner to Professional',
 'description': "A team sport where players aim to score goals by getting the ball into the opposing team's net without using their hands or arms.",
 'playersMax': '22',
 'ageRange': '4+',
 'duration': '90 minutes',
 'equipmentNeeded': 'Soccer ball; Goals; Cleats; Shin guards',
 'objective': 'Score more goals than the opposing team within the allotted time.',
 'skillsDeveloped': 'Teamwork; Coordination; Physical fitness',
 'setupTime': '10-15 minutes',
 'place': 'Soccer field or pitch',
 'physicalIntensityLevel': 'High',
 'educationalBenefits': 'Promotes physical fitness, teamwork, and strategic thinking',
 'category': 'Outdoor Sports and Games'}

In [6]:
import minsearch

# Initialize the Index
index = minsearch.Index(
    text_fields=[
        'gameName', 'alternateNames', 'subcategory', 'description', 'objective', 
        'skillsDeveloped', 'place', 'physicalIntensityLevel', 'educationalBenefits'
    ],
    keyword_fields=[
        'gameId', 'setupTime', 'equipmentNeeded',
        'category', 'ageRange', 'level', 'playersMax', 'duration'
    ]
)

documents = df.to_dict(orient='records')


# Fit the index with documents
index.fit(documents)

# Perform a search
results = index.search(
    query="open-world exploration and puzzle solving",
    filter_dict={'subcategory': 'Action-Adventure', 'indoorOutdoor': 'Indoor'},
    boost_dict={'description': 2, 'objective': 1.5},
    num_results=5
)

# Display results
for doc in results:
    print(f"ID: {doc['gameId']}, Game Name: {doc.get('gameName', 'N/A')}, Objective: {doc['objective']}")

ID: 159, Game Name: Hollow Knight, Objective: Explore the kingdom of Hallownest and uncover its mysteries.
ID: 171, Game Name: The Elder Scrolls V: Skyrim, Objective: Defeat Alduin and complete various quests.
ID: 167, Game Name: Grand Theft Auto V, Objective: Complete missions and explore the open world.
ID: 144, Game Name: The Legend of Zelda: Breath of the Wild, Objective: Defeat Calamity Ganon to save Hyrule.
ID: 163, Game Name: Elden Ring, Objective: Defeat bosses and unravel the mysteries of the Elden Ring.


## RAG flow

In [7]:
import openai
from openai import OpenAI

client = OpenAI()

In [8]:
def search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [9]:
prompt_template = """
You're a game shop assistant. Answer the QUESTION based on the CONTEXT from our games database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

entry_template = """
  'gameName': {gameName}
  'alternateNames': {alternateNames}
  'subcategory': {subcategory}
  'level': {level},
  'description': {description}
  'playersMax': {playersMax}
  'ageRange': {ageRange}
  'duration': {duration}
  'equipmentNeeded': {equipmentNeeded}
  'objective': {objective},
  'skillsDeveloped': {skillsDeveloped},
  'setupTime': {setupTime},
  'place': {place}
  'physicalIntensityLevel': {physicalIntensityLevel}
  'educationalBenefits': {educationalBenefits}
  'category': {category}
""".strip()

def build_prompt(query, search_results):
    context = ""
    
    for doc in search_results:
        context = context + entry_template.format(**doc) + "\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [10]:
def llm(prompt, model='gpt-4o-mini'):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [11]:
def rag(query, model='gpt-4o-mini'):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt, model=model)
    return answer

In [18]:
question = 'Find the game Soccer?'
answer = rag(question)
# Replace '. ' with '.\n' to insert a newline after each sentence
formatted_answer = answer.replace('. ', '.\n')

# Print the formatted answer
print(formatted_answer)

The game you are looking for is **Soccer**.
It is described as a team sport where players aim to score goals by getting the ball into the opposing team's net without using their hands or arms.
It can accommodate up to 22 players, is suitable for ages 4 and up, and typically lasts 90 minutes.
The equipment needed includes a soccer ball, goals, cleats, and shin guards.
The objective of the game is to score more goals than the opposing team within the allotted time.
Soccer promotes teamwork, coordination, and physical fitness.


In [19]:
question = "Tell me about strategy games for two players."
answer = rag(question)
formatted_answer = answer.replace('. ', '.\n')

# Print the formatted answer
print(formatted_answer)

For two players looking for strategy games, here are some options based on the context provided:

1.
**Settlers of Catan**: Although this game can accommodate up to 4 players, it's ideal for two players as well.
In this strategy board game, players collect resources and build settlements to earn victory points, enhancing their skills in resource management and strategic planning.
The game typically lasts 1-2 hours, has a low physical intensity level, and requires a game board, resource cards, tokens, and dice.

2.
**Airsoft**: While not strictly a traditional board game, Airsoft can be played in strategic two-player formats.
This team game involves completing objectives or eliminating opponents using replica firearms that shoot plastic pellets.
It promotes teamwork, strategy, and coordination and can be set up in about 15 minutes with moderate to high physical intensity.

3.
**Mahjong**: This tile game is typically played by 4 players, but it can be adapted for two players.
It involves

In [20]:
question = "What board games help develop strategic thinking?"
answer = rag(question)
print(answer)

The following board games help develop strategic thinking:

1. **Hnefatafl (Viking Chess)** - Develops strategic planning and tactical thinking.
2. **Xiangqi (Chinese Chess)** - Encourages strategic thinking, concentration, and planning.
3. **Go (Weiqi; Baduk)** - Enhances strategic thinking, patience, and spatial reasoning.
4. **Chess (International Chess)** - Develops logical reasoning and strategic planning.
5. **Five Tribes** - Promotes strategic planning, resource management, and tactical thinking.
6. **Sequence** - Enhances strategic thinking, probability, and teamwork.
7. **Dara** - Develops strategic thinking, planning, and concentration.


In [21]:
question = "Recommend a game for 10+ age group that lasts around 30 minutes."
answer = rag(question)
formatted_answer = answer.replace('. ', '.\n')

# Print the formatted answer
print(formatted_answer)


I recommend **Table Football** (also known as Foosball).
It is suitable for ages 6 and up, lasts between 10 to 30 minutes, and is a fun tabletop game that enhances hand-eye coordination, reflexes, and strategy.


In [22]:
question = "Recommend games for 10+ age group that lasts around 30 minutes."
answer = rag(question)
print(answer)

For the 10+ age group with a duration of around 30 minutes, I recommend the following games:

1. **The Resistance**
   - **Age Range**: 13+
   - **Duration**: 30 minutes
   - **Objective**: Complete missions successfully without being sabotaged by spies.
   - **Skills Developed**: Deductive reasoning; Strategy; Communication.

2. **Werewolf**
   - **Age Range**: 13+
   - **Duration**: 30-90 minutes (can be played in shorter sessions)
   - **Objective**: Eliminate the opposing team by deducing roles and voting.
   - **Skills Developed**: Deductive reasoning; Strategy; Social skills.

Both of these games are engaging and suitable for players aged 10 and up, providing a fun challenge within the specified time frame.
